In [67]:
import pandas as pd
import re, os
import os

In [69]:
# path containing gas phase calculations
gas_path = 'emma_davies_gas_output//scratch//chmbnn//mopac_jobs//emma_davies_gas'
# path containing solution phase calculation
sol_path = 'emma_davies_sol_output//scratch//chmbnn//mopac_jobs//emma_davies_sol'
#path containing solvent information
solvent_path = 'emma_davies_solvents_out//emma_davies_solvents'

## Extracting thermodynamic values from .aux files

In [72]:
# extract enthalpy and entropy at 298K from .out files to begin with
# this will take advatange of the fact that the values at 298K are the
# first ones listed in the thermodynamics section

df_thermo_gas = pd.DataFrame(columns=['Compound', 'H_gas', 'S_gas'])

for filename in os.listdir(gas_path):
    if filename.endswith(".aux"):  # Process .aux files only
        # Extract compound name from filename
        compound = filename.replace('MNDO_', '').replace('.aux', '').replace('_gas','')
        full_filename = os.path.join(gas_path, filename)

        enthalpy = None
        entropy_298K = None

        with open(full_filename, 'r') as f:
            for line in f:
                if "HEAT_OF_FORM" in line:  # Extract enthalpy
                    enthalpy_str = line.split('=')[-1].strip().replace('D', 'E')  # Fix notation
                    enthalpy = float(enthalpy_str)  # Convert to float
                
                if "ENTROPY_TOT" in line:
                    entropy_values = next(f).split()  # Read next line with values
                    if entropy_values:
                        entropy_298K_str = entropy_values[0].replace('D', 'E')  # Fix notation
                        entropy_298K = float(entropy_298K_str)  # Convert to float

                # Stop reading once both values are found
                if enthalpy is not None and entropy_298K is not None:
                    break

        # Add to DataFrame only if both values were found
        if enthalpy is not None and entropy_298K is not None:
            df_thermo_gas = pd.concat([df_thermo_gas, pd.DataFrame([{
                'Compound': compound, 'H_gas': enthalpy, 'S_gas': entropy_298K
            }])], ignore_index=True)


print(df_thermo_gas)

C:\Users\emmad\AppData\Local\Temp\ipykernel_17428\4156628758.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_thermo_gas = pd.concat([df_thermo_gas, pd.DataFrame([{


                                          Compound    H_gas    S_gas
0            AAOVKJBEBIDNHE-UHFFFAOYNA-N_1-octanol  44.6207  132.314
1              AAOVKJBEBIDNHE-UHFFFAOYNA-N_ethanol  44.6207  132.314
2     AAOVKJBEBIDNHE-UHFFFAOYNA-N_propylene_glycol  44.6207  132.314
3                AAOVKJBEBIDNHE-UHFFFAOYNA-N_water  44.6207  132.314
4         ACWBQPMHZXGDFX-XGLYSIDGNA-N_acetonitrile   8.8862  196.612
...                                            ...      ...      ...
3409             ZWLPBLYKEWSWPD-BGGKNDAXNA-N_water -60.3937   89.771
3410         ZZUFCTLCJUWOSV-CDZRGBSPNA-N_1-octanol  41.7013  138.972
3411           ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethanol  41.7013  138.972
3412     ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethyl_acetate  41.7013  138.972
3413             ZZUFCTLCJUWOSV-CDZRGBSPNA-N_water  41.7013  138.972

[3414 rows x 3 columns]


In [74]:
# extract enthalpy and entropy at 298K from .out files to begin with
# this will take advatange of the fact that the values at 298K are the
# first ones listed in the thermodynamics section

df_thermo_sol = pd.DataFrame(columns = ['Compound', 'H_sol', 'S_sol'])

for filename in os.listdir(sol_path):
    if filename.endswith(".aux"):  # Process .aux files instead of .out files
        # Extract compound name from filename
        compound = filename.replace('MNDO_', '').replace('.aux', '')
        full_filename = os.path.join(sol_path, filename)

        enthalpy = None
        entropy_298K = None
        
        with open(full_filename, 'r') as f:
            for line in f:
                if "HEAT_OF_FORM" in line:  # Extract enthalpy
                    enthalpy_str = line.split('=')[-1].strip().replace('D', 'E')  # Fix notation
                    enthalpy = float(enthalpy_str)  # Convert to float
                
                if "ENTROPY_TOT" in line:
                    entropy_values = next(f).split()  # Read next line with values
                    if entropy_values:
                        entropy_298K_str = entropy_values[0].replace('D', 'E')  # Fix notation
                        entropy_298K = float(entropy_298K_str)  # Convert to float

                # Stop reading once both values are found
                if enthalpy is not None and entropy_298K is not None:
                    break

        # Add to DataFrame only if both values were found
        if enthalpy is not None and entropy_298K is not None:
            df_thermo_sol = pd.concat([df_thermo_sol, pd.DataFrame([{
                'Compound': compound, 'H_sol': enthalpy, 'S_sol': entropy_298K
            }])], ignore_index=True)

print(df_thermo_sol)

C:\Users\emmad\AppData\Local\Temp\ipykernel_17428\1015066189.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_thermo_sol = pd.concat([df_thermo_sol, pd.DataFrame([{


                                          Compound     H_sol    S_sol
0            AAOVKJBEBIDNHE-UHFFFAOYNA-N_1-octanol  37.50550  133.406
1              AAOVKJBEBIDNHE-UHFFFAOYNA-N_ethanol  36.69780  133.677
2     AAOVKJBEBIDNHE-UHFFFAOYNA-N_propylene_glycol  36.60630  133.723
3                AAOVKJBEBIDNHE-UHFFFAOYNA-N_water  36.29080  133.498
4         ACWBQPMHZXGDFX-XGLYSIDGNA-N_acetonitrile  -8.97366  196.238
...                                            ...       ...      ...
3408             ZWLPBLYKEWSWPD-BGGKNDAXNA-N_water -66.85890   90.252
3409         ZZUFCTLCJUWOSV-CDZRGBSPNA-N_1-octanol  16.02550  139.299
3410           ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethanol  13.02470  139.400
3411     ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethyl_acetate  19.26370  139.090
3412             ZZUFCTLCJUWOSV-CDZRGBSPNA-N_water  11.50730  139.475

[3413 rows x 3 columns]


In [76]:
print(df_thermo_sol.columns)
print(df_thermo_gas.columns)

Index(['Compound', 'H_sol', 'S_sol'], dtype='object')
Index(['Compound', 'H_gas', 'S_gas'], dtype='object')


In [78]:
df_thermo_gas['Compound'] = df_thermo_gas['Compound'].str.strip()
df_thermo_sol['Compound'] = df_thermo_sol['Compound'].str.strip()

# merging dataframes from gas and solution calculations
df_thermo = pd.merge(df_thermo_gas, df_thermo_sol, on='Compound')

print(df_thermo)

                                          Compound    H_gas    S_gas  \
0            AAOVKJBEBIDNHE-UHFFFAOYNA-N_1-octanol  44.6207  132.314   
1              AAOVKJBEBIDNHE-UHFFFAOYNA-N_ethanol  44.6207  132.314   
2     AAOVKJBEBIDNHE-UHFFFAOYNA-N_propylene_glycol  44.6207  132.314   
3                AAOVKJBEBIDNHE-UHFFFAOYNA-N_water  44.6207  132.314   
4         ACWBQPMHZXGDFX-XGLYSIDGNA-N_acetonitrile   8.8862  196.612   
...                                            ...      ...      ...   
3408             ZWLPBLYKEWSWPD-BGGKNDAXNA-N_water -60.3937   89.771   
3409         ZZUFCTLCJUWOSV-CDZRGBSPNA-N_1-octanol  41.7013  138.972   
3410           ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethanol  41.7013  138.972   
3411     ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethyl_acetate  41.7013  138.972   
3412             ZZUFCTLCJUWOSV-CDZRGBSPNA-N_water  41.7013  138.972   

         H_sol    S_sol  
0     37.50550  133.406  
1     36.69780  133.677  
2     36.60630  133.723  
3     36.29080  133.498  
4    

In [80]:
# calculate G_sol and DeltaG_sol descriptors, and remove the rest
# of the data from df_thermo

df_thermo['H_gas'] = df_thermo['H_gas'].astype(float)
df_thermo['S_gas'] = df_thermo['S_gas'].astype(float)
df_thermo['H_sol'] = df_thermo['H_sol'].astype(float)
df_thermo['S_sol'] = df_thermo['S_sol'].astype(float)
df_thermo['G_gas'] = df_thermo['H_gas']-(df_thermo['S_gas']*298)
df_thermo['G_sol'] = df_thermo['H_sol']-(df_thermo['S_sol']*298)
df_thermo['DeltaG_sol'] = df_thermo['G_sol']-df_thermo['G_gas']
df_thermo.drop(['H_gas', 'S_gas', 'H_sol', 'S_sol', 'G_gas'], axis = 1)

,Compound,G_sol,DeltaG_sol
0,AAOVKJBEBIDNHE-UHFFFAOYNA-N_1-octanol,-39717.48250,-332.53120
1,AAOVKJBEBIDNHE-UHFFFAOYNA-N_ethanol,-39799.04820,-414.09690
2,AAOVKJBEBIDNHE-UHFFFAOYNA-N_propylene_glycol,-39812.84770,-427.89640
3,AAOVKJBEBIDNHE-UHFFFAOYNA-N_water,-39746.11320,-361.16190
4,ACWBQPMHZXGDFX-XGLYSIDGNA-N_acetonitrile,-58487.89766,93.59214
...,...,...,...
3408,ZWLPBLYKEWSWPD-BGGKNDAXNA-N_water,-26961.95490,-149.80320
3409,ZZUFCTLCJUWOSV-CDZRGBSPNA-N_1-octanol,-41495.07650,-123.12180
3410,ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethanol,-41528.17530,-156.22060
3411,ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethyl_acetate,-41429.55630,-57.60160


## Collect volume (not available in PM3) and dipole moment from .aux files

In [83]:
# will read straight from solution structures and bypass gas phase structures

import pandas as pd

# Create the dataframe to store results
df_dip_vol = pd.DataFrame(columns=['Compound', 'volume', 'sol_dip'])

# Loop over your files in sol_path
for filename in os.listdir(sol_path):
    if '.aux' in filename:
        # Get InChi code from filename
        compound = filename.replace('MNDO_', '')
        compound = compound.replace('.aux', '')
        
        # Open aux file
        full_filename = os.path.join(sol_path, filename)
        f = open(full_filename, 'r')
        
        # Initialize variables
        sol_dip = None
        volume = None
        
        # Find thermodynamic values
        for line in f:
            if 'DIPOLE' in line:
                dipole_line = line.split('=')
                sol_dip_string = dipole_line[1].replace('\n', '')
                sol_dip_string = sol_dip_string.replace('+', '', 1)
                sol_dip_string_split = sol_dip_string.split('D')
                sol_dip = float(sol_dip_string_split[0]) * pow(10, float(sol_dip_string_split[1]))
            
            if 'VOLUME' in line:
                volume_line = line.split('=')
                volume_string = volume_line[1].replace('\n', '')
                volume_string = volume_string.replace('+', '', 1)
                volume_string_split = volume_string.split('D')
                volume = float(volume_string_split[0]) * pow(10, float(volume_string_split[1]))

        # Save extracted data to dataframe
        new_row = pd.DataFrame({'Compound': [compound], 'volume': [volume], 'sol_dip': [sol_dip]})
        df_dip_vol = pd.concat([df_dip_vol, new_row], ignore_index=True)

# Display the resulting DataFrame
print(df_dip_vol)


C:\Users\emmad\AppData\Local\Temp\ipykernel_17428\1790697880.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_dip_vol = pd.concat([df_dip_vol, new_row], ignore_index=True)


                                          Compound   volume   sol_dip
0            AAOVKJBEBIDNHE-UHFFFAOYNA-N_1-octanol  327.610   4.16476
1              AAOVKJBEBIDNHE-UHFFFAOYNA-N_ethanol  327.610   4.33761
2     AAOVKJBEBIDNHE-UHFFFAOYNA-N_propylene_glycol  327.610   4.35765
3                AAOVKJBEBIDNHE-UHFFFAOYNA-N_water  327.610   4.42740
4         ACWBQPMHZXGDFX-XGLYSIDGNA-N_acetonitrile  550.414   7.71316
...                                            ...      ...       ...
3408             ZWLPBLYKEWSWPD-BGGKNDAXNA-N_water  166.628   2.93398
3409         ZZUFCTLCJUWOSV-CDZRGBSPNA-N_1-octanol  343.778  11.34140
3410           ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethanol  343.778  11.66230
3411     ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethyl_acetate  343.778  10.99980
3412             ZZUFCTLCJUWOSV-CDZRGBSPNA-N_water  343.778  11.82610

[3413 rows x 3 columns]


## Reading HOMO and LUMO energy of the gas phase structures

In [86]:
# need to extract HOMO and LUMO of the gas phase structure

# Assign base values for HOMO and LUMO of solvent


In [17]:
import os

solvent_name_file = 'unique_solvent_names.csv'

if os.path.exists(solvent_name_file):
    print(f"File '{solvent_name_file}' found.")
else:
    print(f"Error: File '{solvent_name_file}' not found!")

File 'unique_solvent_names.csv' found.


In [35]:
import pandas as pd
import os

# Create the DataFrame for HOMO and LUMO values
df_homo_lumo = pd.DataFrame(columns=['Compound', 'HOMO', 'LUMO'])

solvents_df = pd.read_csv(solvent_name_file) 
solvent_list = solvents_df['solvent_name'].dropna().astype(str).tolist()  

# Loop through files in the specified directory
for filename in os.listdir(sol_path):
    if filename.endswith('.aux'):
        # Remove the prefix and suffix
        compound = filename.replace('MNDO_', '').replace('.aux', '')

       # Identify and remove the solvent name (it appears before .aux)
        for solvent in solvent_list:
            if compound.endswith(f"_{solvent}"):  # Check if it ends with "_solvent"
                compound = compound[: -len(solvent) - 1]  # Remove the solvent and the underscore

        
        # Open the .aux file
        full_filename = os.path.join(sol_path, filename)
        with open(full_filename, 'r') as f:
            lines = f.read().split('\n')
        
        # Find data for Molecular Orbital energies and occupancies
        for line in lines:
            if 'EIGENVALUES' in line:
                index = lines.index(line)
        
        MOs = []
        index = index + 1
        while 'MOLECULAR_ORBITAL_OCCUPANCIES' not in lines[index]:
            MO_line = lines[index].strip()  # Strip any leading/trailing whitespace
            index += 1
            MO_line_split = MO_line.split()
            MOs.extend(MO_line_split)
        
        index += 1
        occupancies = []
        while '###' not in lines[index]:
            occupancy_line = lines[index].strip()
            index += 1
            occupancy_line_split = occupancy_line.split()
            occupancies.extend(occupancy_line_split)
        
        # Find HOMO and LUMO energies solute
        lumo_index = occupancies.index('0.0000')
        homo_index = lumo_index - 1
        homo = MOs[homo_index]
        lumo = MOs[lumo_index]
        
        # Create a DataFrame for the new row
        new_row = pd.DataFrame({'Compound': [compound], 'HOMO': [homo], 'LUMO': [lumo]})
        
        # Use pd.concat to append the new row
        df_homo_lumo = pd.concat([df_homo_lumo, new_row], ignore_index=True)

# Display the resulting DataFrame
print(df_homo_lumo)


                         Compound    HOMO    LUMO
0     AAOVKJBEBIDNHE-UHFFFAOYNA-N  -9.507  -0.782
1     AAOVKJBEBIDNHE-UHFFFAOYNA-N  -9.499  -0.772
2     AAOVKJBEBIDNHE-UHFFFAOYNA-N  -9.498  -0.771
3     AAOVKJBEBIDNHE-UHFFFAOYNA-N  -9.495  -0.767
4     ACWBQPMHZXGDFX-XGLYSIDGNA-N  -9.123  -0.488
...                           ...     ...     ...
3408  ZWLPBLYKEWSWPD-BGGKNDAXNA-N  -9.608  -0.543
3409  ZZUFCTLCJUWOSV-CDZRGBSPNA-N  -9.305  -2.126
3410  ZZUFCTLCJUWOSV-CDZRGBSPNA-N  -9.281  -2.130
3411  ZZUFCTLCJUWOSV-CDZRGBSPNA-N  -9.330  -2.120
3412  ZZUFCTLCJUWOSV-CDZRGBSPNA-N  -9.269  -2.132

[3413 rows x 3 columns]


In [37]:
# Create the DataFrame for HOMO and LUMO values
df_homo_lumo_solvent = pd.DataFrame(columns=['Solvent', 'HOMO_solvent', 'LUMO_solvent'])

# Loop through files in the solvent
for filename in os.listdir(solvent_path):
    if '.aux' in filename:
        # Extract the compound name from the filename
        compound = filename.replace('MNDO_', '')
        compound = compound.replace('.aux', '')
        
        # Open the .aux file
        full_filename = os.path.join(solvent_path, filename)
        with open(full_filename, 'r') as f:
            lines = f.read().split('\n')
        
        # Find data for Molecular Orbital energies and occupancies
        for line in lines:
            if 'EIGENVALUES' in line:
                index = lines.index(line)
        
        MOs = []
        index = index + 1
        while 'MOLECULAR_ORBITAL_OCCUPANCIES' not in lines[index]:
            MO_line = lines[index].strip()  # Strip any leading/trailing whitespace
            index += 1
            MO_line_split = MO_line.split()
            MOs.extend(MO_line_split)
        
        index += 1
        occupancies = []
        while '###' not in lines[index]:
            occupancy_line = lines[index].strip()
            index += 1
            occupancy_line_split = occupancy_line.split()
            occupancies.extend(occupancy_line_split)
        
        # Find HOMO and LUMO energies solute
        lumo_index = occupancies.index('0.0000')
        homo_index = lumo_index - 1
        homo = MOs[homo_index]
        lumo = MOs[lumo_index]
        
        # Create a DataFrame for the new row
        new_row = pd.DataFrame({'Solvent': [compound], 'HOMO_solvent': [homo], 'LUMO_solvent': [lumo]})
        
        # Use pd.concat to append the new row
        df_homo_lumo_solvent = pd.concat([df_homo_lumo_solvent, new_row], ignore_index=True)

# Display the resulting DataFrame
print(df_homo_lumo_solvent)

               Solvent HOMO_solvent LUMO_solvent
0   1,2-dichloroethane      -12.363        0.082
1      1,2-propanediol      -10.856        3.195
2          1,4-dioxane      -10.897        2.931
3            1-butanol      -11.192        3.263
4       1-chlorobutane      -12.000        1.018
..                 ...          ...          ...
71                 THF      -10.859        3.088
72             toluene       -9.330        0.276
73    trifluoroethanol      -12.480        1.879
74            undecane      -11.764        3.315
75               water      -12.145        4.945

[76 rows x 3 columns]


In [109]:
# Create DataFrame for results
df_homo_lumo_results = pd.DataFrame(columns=['Solute', 'Solvent', 'Lsolu_Hsolv', 'Lsolv_Hsolu'])

# Loop through files in the specified directory
for filename in os.listdir(gas_path):
    if filename.endswith('.aux'):
        # Remove prefix and suffix
        compound = filename.replace('MNDO_', '').replace('_gas.aux', '')

        # Identify solute and solvent by splitting at the first underscore only
        parts = compound.split('_', 1)
        if len(parts) < 2:
            continue  # Skip if not in 'solute_solvent' format

        solute = parts[0].strip()  # Remove any leading/trailing spaces
        solvent = parts[1].strip()  # Remove any leading/trailing spaces

        # Check if solute and solvent exist in respective DataFrames
        if solute in df_homo_lumo['Compound'].values and solvent in df_homo_lumo_solvent['Solvent'].values:

            try:
                # Ensure correct column names and convert to numeric types
                solute_HOMO = pd.to_numeric(df_homo_lumo.loc[df_homo_lumo['Compound'] == solute, 'HOMO'].values[0], errors='coerce')
                solute_LUMO = pd.to_numeric(df_homo_lumo.loc[df_homo_lumo['Compound'] == solute, 'LUMO'].values[0], errors='coerce')
                solvent_HOMO = pd.to_numeric(df_homo_lumo_solvent.loc[df_homo_lumo_solvent['Solvent'] == solvent, 'HOMO_solvent'].values[0], errors='coerce')
                solvent_LUMO = pd.to_numeric(df_homo_lumo_solvent.loc[df_homo_lumo_solvent['Solvent'] == solvent, 'LUMO_solvent'].values[0], errors='coerce')

                # Check if any of the values are NaN (conversion failed)
                if any(pd.isna(value) for value in [solute_HOMO, solute_LUMO, solvent_HOMO, solvent_LUMO]):
                    print(f"Skipping: Invalid numeric values for {solute} and {solvent}")
                    continue

                # Calculate descriptors
                Lsolu_Hsolv = solute_LUMO - solvent_HOMO
                Lsolv_Hsolu = solvent_LUMO - solute_HOMO

                # Create a temporary DataFrame for this pair and concatenate it
                temp_df = pd.DataFrame({'Solute': [solute], 'Solvent': [solvent], 
                                        'Lsolu_Hsolv': [Lsolu_Hsolv], 'Lsolv_Hsolu': [Lsolv_Hsolu]})

                # Concatenate the results to the main DataFrame
                df_homo_lumo_results = pd.concat([df_homo_lumo_results, temp_df], ignore_index=True)
                
            except KeyError as e:
                print(f"KeyError occurred: {e}")
                print(f"Solute: {solute}, Solvent: {solvent}")
                print(f"Available columns in df_homo_lumo_solvent: {df_homo_lumo_solvent.columns}")

df_homo_lumo_results['Compound'] = df_homo_lumo_results['Solute'] + '_' + df_homo_lumo_results['Solvent']
df_homo_lumo_results = df_homo_lumo_results.drop(['Solute', 'Solvent'], axis=1)

# Reorder columns to make 'compound' the first column and drop the extra 'compound' column (if any)
# Ensure that only one 'compound' column is present by dropping any redundant 'compound' columns
df_homo_lumo_results = df_homo_lumo_results.loc[:, ~df_homo_lumo_results.columns.duplicated()]

# Move the 'compound' column to the first position
df_homo_lumo_results = df_homo_lumo_results[['Compound'] + [col for col in df_homo_lumo_results.columns if col != 'Compound']]

print(df_homo_lumo_results)


C:\Users\emmad\AppData\Local\Temp\ipykernel_17428\3074461285.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_homo_lumo_results = pd.concat([df_homo_lumo_results, temp_df], ignore_index=True)


                                          Compound  Lsolu_Hsolv  Lsolv_Hsolu
0            AAOVKJBEBIDNHE-UHFFFAOYNA-N_1-octanol       10.403       12.696
1              AAOVKJBEBIDNHE-UHFFFAOYNA-N_ethanol       10.539       12.888
2     AAOVKJBEBIDNHE-UHFFFAOYNA-N_propylene_glycol        9.972       12.713
3                AAOVKJBEBIDNHE-UHFFFAOYNA-N_water       11.363       14.452
4         ACWBQPMHZXGDFX-XGLYSIDGNA-N_acetonitrile       12.353       10.775
...                                            ...          ...          ...
3398             ZWLPBLYKEWSWPD-BGGKNDAXNA-N_water       11.614       14.582
3399         ZZUFCTLCJUWOSV-CDZRGBSPNA-N_1-octanol        9.059       12.494
3400           ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethanol        9.195       12.686
3401     ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethyl_acetate        9.212        9.990
3402             ZZUFCTLCJUWOSV-CDZRGBSPNA-N_water       10.019       14.250

[3403 rows x 3 columns]


## Charge descriptors time
These are taken from the solution structure calculations. The descriptors are:
* O_charges: sum of charges on oxygen atoms
* C_charges: sum of charges on carbon atoms
* Most_neg: charge on most negative atom
* Most_pos: charge on most positive atom
* Het_charges: sum of charges on solution structure non-hydrogen/carbon atoms

The strategy is to identify the list of charges and the list of atoms, and calculate from there.

In [97]:
import pandas as pd
import os

def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

df_charges = pd.DataFrame(columns = ['Compound', 'O_charges', 'C_charges', 'Most_neg', 'Most_pos', 'Het_charges'])

# read .aux files
for filename in os.listdir(sol_path):
    if '.aux' in filename:
        
        # get InChi code from filename
        compound = filename.replace('MNDO_','')
        compound = compound.replace('.aux','')
        
        # open output file and read the content
        full_filename = os.path.join(sol_path, filename)
        with open(full_filename, 'r') as f:
            lines = f.read().split('\n')
        f.close()
        
        # find charge data
        for line in lines:
            if 'ATOM_CHARGES' in line:
                index = lines.index(line)
        charges = []
        index = index + 1
        while 'GRADIENTS' not in lines[index]:
            charge_line = lines[index]
            charge_line = charge_line.replace('\n','')
            index = index + 1
            charge_line_split = charge_line.split()
            charges = charges + charge_line_split
        
        # convert charges to number
        charges_number = [float(x) for x in charges if x.replace('.', '', 1).replace('-', '', 1).isdigit()]
        
        # find atom list
        for line in lines:
            if 'ATOM_EL' in line:
                index = lines.index(line)
        atoms = []
        index = index + 1
        while 'ATOM_CORE' not in lines[index]:
            atom_line = lines[index]
            atom_line = atom_line.replace('\n','')
            index = index + 1
            atom_line_split = atom_line.split()
            atoms = atoms + atom_line_split
        
        # calculate descriptors
        O_charges = 0
        O_indexes  = get_index_positions(atoms, 'O')
        for index in O_indexes:
            O_charges = O_charges + charges_number[index]
        C_charges = 0
        C_indexes  = get_index_positions(atoms, 'C')
        for index in C_indexes:
            C_charges = C_charges + charges_number[index]
        Most_pos = max(charges_number)
        Most_neg = min(charges_number)
        Het_charges = 0
        list_length = len(atoms)
        for index in range(0, list_length):
            if atoms[index] != 'C' and atoms[index] != 'H':
                Het_charges = Het_charges + charges_number[index]
        # save descriptors
        new_row = {'Compound':compound, 'O_charges':O_charges, 'C_charges':C_charges, 'Most_neg':Most_neg, 'Most_pos':Most_pos, 'Het_charges':Het_charges}
        df_charges = pd.concat([df_charges, pd.DataFrame([new_row])], ignore_index=True)
print(df_charges)

C:\Users\emmad\AppData\Local\Temp\ipykernel_17428\3307558102.py:83: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_charges = pd.concat([df_charges, pd.DataFrame([new_row])], ignore_index=True)


                                          Compound  O_charges  C_charges  \
0            AAOVKJBEBIDNHE-UHFFFAOYNA-N_1-octanol   -0.11479    3.69845   
1              AAOVKJBEBIDNHE-UHFFFAOYNA-N_ethanol   -0.11683    3.68268   
2     AAOVKJBEBIDNHE-UHFFFAOYNA-N_propylene_glycol   -0.11706    3.68092   
3                AAOVKJBEBIDNHE-UHFFFAOYNA-N_water   -0.11785    3.67476   
4         ACWBQPMHZXGDFX-XGLYSIDGNA-N_acetonitrile    1.70314   -0.53088   
...                                            ...        ...        ...   
3408             ZWLPBLYKEWSWPD-BGGKNDAXNA-N_water   -0.57086    4.45276   
3409         ZZUFCTLCJUWOSV-CDZRGBSPNA-N_1-octanol    0.71796    5.65135   
3410           ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethanol    0.70972    5.64503   
3411     ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethyl_acetate    0.72709    5.65839   
3412             ZZUFCTLCJUWOSV-CDZRGBSPNA-N_water    0.70563    5.64191   

      Most_neg  Most_pos  Het_charges  
0     -0.41832   1.98510      0.86870  
1     -

## list of dataframe to recombine
* df_thermo
* df_dip_vol
* df_homo_lumo_results
* df_charges

In [111]:
# the dataframes will be combined into one now. It will be convenient to 
# add the MW and SASA data 
df_thermo_dip_vol = pd.merge(df_thermo, df_dip_vol, on='Compound')
df_thermo_dip_vol_homo_lumo = pd.merge(df_thermo_dip_vol, df_homo_lumo_results, on='Compound')
df_all = pd.merge(df_thermo_dip_vol_homo_lumo, df_charges, on='Compound')

In [113]:
df_all

,Compound,H_gas,S_gas,H_sol,S_sol,G_gas,G_sol,DeltaG_sol,volume,sol_dip,Lsolu_Hsolv,Lsolv_Hsolu,O_charges,C_charges,Most_neg,Most_pos,Het_charges
0,AAOVKJBEBIDNHE-UHFFFAOYNA-N_1-octanol,44.6207,132.314,37.50550,133.406,-39384.9513,-39717.48250,-332.53120,327.610,4.16476,10.403,12.696,-0.11479,3.69845,-0.41832,1.98510,0.86870
1,AAOVKJBEBIDNHE-UHFFFAOYNA-N_ethanol,44.6207,132.314,36.69780,133.677,-39384.9513,-39799.04820,-414.09690,327.610,4.33761,10.539,12.888,-0.11683,3.68268,-0.42737,1.98521,0.86919
2,AAOVKJBEBIDNHE-UHFFFAOYNA-N_propylene_glycol,44.6207,132.314,36.60630,133.723,-39384.9513,-39812.84770,-427.89640,327.610,4.35765,9.972,12.713,-0.11706,3.68092,-0.42840,1.98523,0.86926
3,AAOVKJBEBIDNHE-UHFFFAOYNA-N_water,44.6207,132.314,36.29080,133.498,-39384.9513,-39746.11320,-361.16190,327.610,4.42740,11.363,14.452,-0.11785,3.67476,-0.43195,1.98527,0.86950
4,ACWBQPMHZXGDFX-XGLYSIDGNA-N_acetonitrile,8.8862,196.612,-8.97366,196.238,-58581.4898,-58487.89766,93.59214,550.414,7.71316,12.353,10.775,1.70314,-0.53088,-0.46885,1.89511,0.88377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3397,ZWLPBLYKEWSWPD-BGGKNDAXNA-N_water,-60.3937,89.771,-66.85890,90.252,-26812.1517,-26961.95490,-149.80320,166.628,2.93398,11.614,14.582,-0.57086,4.45276,-0.46143,1.91230,-0.57086
3398,ZZUFCTLCJUWOSV-CDZRGBSPNA-N_1-octanol,41.7013,138.972,16.02550,139.299,-41371.9547,-41495.07650,-123.12180,343.778,11.34140,9.059,12.494,0.71796,5.65135,-0.75321,1.98329,-0.74858
3399,ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethanol,41.7013,138.972,13.02470,139.400,-41371.9547,-41528.17530,-156.22060,343.778,11.66230,9.195,12.686,0.70972,5.64503,-0.76430,1.98319,-0.76759
3400,ZZUFCTLCJUWOSV-CDZRGBSPNA-N_ethyl_acetate,41.7013,138.972,19.26370,139.090,-41371.9547,-41429.55630,-57.60160,343.778,10.99980,9.212,9.990,0.72709,5.65839,-0.74115,1.98340,-0.72780


In [115]:
df_all.to_csv('MNDO_descriptors.csv', index=False)

In [135]:
from rdkit import Chem
from rdkit.Chem import Descriptors
import pandas as pd

# Load the DataFrame
filtered_data_updated_final2 = pd.read_csv('filtered_data_updated_final2.csv')

# Function to calculate molecular weight using RDKit
def get_molecular_weight(smiles):
    mol = Chem.MolFromSmiles(smiles)  # Convert SMILES string to molecule
    if mol:
        return Descriptors.MolWt(mol)  # Return the molecular weight
    else:
        return None  # Return None if the SMILES is invalid

# Filter the rows where MW is missing (NaN) and solute_smiles are available
filtered_data_updated_final2['MW'] = filtered_data_updated_final2.apply(
    lambda row: get_molecular_weight(row['solute_smiles']) if pd.isnull(row['MW']) else row['MW'], axis=1
)

# Save the updated DataFrame with MW values
filtered_data_updated_final2.to_csv('filtered_data_updated_final2_with_MW.csv', index=False)

# Print the updated DataFrame to verify
print(filtered_data_updated_final2)

         solute_name           solute_smiles     solvent_name  \
0     1,2,4-triazole               C1=CN=NN1       1-propanol   
1     1,2,4-triazole               C1=CN=NN1       2-propanol   
2     1,2,4-triazole               C1=CN=NN1    butyl acetate   
3     1,2,4-triazole               C1=CN=NN1          ethanol   
4     1,2,4-triazole               C1=CN=NN1    ethyl acetate   
...              ...                     ...              ...   
3719        xanthene  c1ccc2c(c1)Cc3ccccc3O2  2-butoxyethanol   
3720        xanthene  c1ccc2c(c1)Cc3ccccc3O2  2-ethoxyethanol   
3721        xanthene  c1ccc2c(c1)Cc3ccccc3O2     acetonitrile   
3722        xanthine  c1[nH]c2c(n1)nc(nc2O)O           PEG400   
3723        xanthine  c1[nH]c2c(n1)nc(nc2O)O            water   

                      solvent_smiles              solute_inchikey       S_M  \
0                               CCCO  NSPMIYGKQJPBQR-TULZNQERNA-N  2.326000   
1                             CC(O)C  NSPMIYGKQJPBQR-TULZNQER

[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[12:42:35] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[12:42:35] Can't kekulize mol.  Unkekulized a

In [146]:
filtered_data_updated_final2_with_MW = pd.read_csv('filtered_data_updated_final2_with_MW.csv')

# Filter rows where MW is missing (NaN)
missing_mw_rows = filtered_data_updated_final2_with_MW[filtered_data_updated_final2_with_MW['MW'].isnull()]

# Print the compound names (or identifiers) for these rows
print(missing_mw_rows['solute_name'])  # Replace 'compound' with the actual column name containing the compound names


377            1,2,4-triazole
378            1,2,4-triazole
379            1,2,4-triazole
380            1,2,4-triazole
381            1,2,4-triazole
382            1,2,4-triazole
383            1,2,4-triazole
384            1,2,4-triazole
385            1,2,4-triazole
386            1,2,4-triazole
400     2-methylbenzimidazole
401     2-methylbenzimidazole
402     2-methylbenzimidazole
403     2-methylbenzimidazole
404     2-methylbenzimidazole
405     2-methylbenzimidazole
406     2-methylbenzimidazole
407     2-methylbenzimidazole
499              azathioprine
500              azathioprine
606              lansoprazole
607              lansoprazole
608              lansoprazole
609              lansoprazole
610              lansoprazole
611              lansoprazole
612              lansoprazole
613              lansoprazole
3668                valsartan
Name: solute_name, dtype: object


In [148]:
# Read the data from the CSV
filtered_data_updated_final2_with_MW = pd.read_csv('filtered_data_updated_final2_with_MW.csv')

# Define a dictionary to map compound names to their MW values
compound_to_mw = {
    '1,2,4-triazole': 69.07,
    '2-methylbenzimidazole': 132.16,
    'azthioprine': 277.27,
    'lansoprazole': 369.4,
    'valsartan': 435.5
}

# Check the column names in case you need to adjust them
print(filtered_data_updated_final2_with_MW.columns)

# Iterate through the DataFrame and update MW based on compound names
for index, row in filtered_data_updated_final2_with_MW.iterrows():
    compound_name = row['solute_name']  # Replace with the actual column name containing the compound name
    if compound_name in compound_to_mw and pd.isnull(row['MW']):
        # Update the MW value if the compound name matches and MW is missing
        filtered_data_updated_final2_with_MW.at[index, 'MW'] = compound_to_mw[compound_name]

# Save the updated DataFrame to a new CSV file
filtered_data_updated_final2_with_MW.to_csv('filtered_data_updated_final2_with_MW2.csv', index=False)

Index(['solute_name', 'solute_smiles', 'solvent_name', 'solvent_smiles',
       'solute_inchikey', 'S_M', 'logS_M', 'MW'],
      dtype='object')


In [152]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors

# Function to calculate SASA for each solute based on its SMILES
def calculate_sasa(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None  # Handle invalid SMILES
    AllChem.Compute2DCoords(mol)
    sasa = rdMolDescriptors.CalcTPSA(mol)  # Topological Polar Surface Area (TPSA) as a proxy for SASA
    return sasa

# Load your DataFrame (assuming it's already loaded)
# filtered_data_updated_final2_with_MW = pd.read_csv('path_to_your_file.csv')  # Uncomment if you need to load the file

# Apply SASA calculation for each solute in the 'solute_smiles' column and add it to a new column 'SASA'
filtered_data_updated_final2_with_MW['SASA'] = filtered_data_updated_final2_with_MW['solute_smiles'].apply(calculate_sasa)

# Save the updated DataFrame with the SASA values to a new CSV file
filtered_data_updated_final2_with_MW.to_csv('filtered_data_with_SASA.csv', index=False)

# Print a confirmation message
print("The updated data with SASA values has been saved to 'filtered_data_with_SASA.csv'.")


[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[14:24:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[14:24:36] Can't kekulize mol.  Unkekulized a

The updated data with SASA values has been saved to 'filtered_data_with_SASA.csv'.


[14:24:37] Can't kekulize mol.  Unkekulized atoms: 22 23 24 25 26


In [154]:
filtered_data_with_SASA = pd.read_csv('filtered_data_with_SASA.csv')

# Filter rows where MW is missing (NaN)
missing_sasa_rows = filtered_data_with_SASA[filtered_data_with_SASA['SASA'].isnull()]

# Print the compound names (or identifiers) for these rows
print(missing_mw_rows['solute_name'])  # Replace 'compound' with the actual column name containing the compound names


377            1,2,4-triazole
378            1,2,4-triazole
379            1,2,4-triazole
380            1,2,4-triazole
381            1,2,4-triazole
382            1,2,4-triazole
383            1,2,4-triazole
384            1,2,4-triazole
385            1,2,4-triazole
386            1,2,4-triazole
400     2-methylbenzimidazole
401     2-methylbenzimidazole
402     2-methylbenzimidazole
403     2-methylbenzimidazole
404     2-methylbenzimidazole
405     2-methylbenzimidazole
406     2-methylbenzimidazole
407     2-methylbenzimidazole
499              azathioprine
500              azathioprine
606              lansoprazole
607              lansoprazole
608              lansoprazole
609              lansoprazole
610              lansoprazole
611              lansoprazole
612              lansoprazole
613              lansoprazole
3668                valsartan
Name: solute_name, dtype: object


In [156]:
# Read the data from the CSV
filtered_data_with_SASA = pd.read_csv('filtered_data_with_SASA.csv')

# Define a dictionary to map compound names to their SASA values
compound_to_SASA = {
    '1,2,4-triazole': 41.6,
    '2-methylbenzimidazole': 28.7,
    'azthioprine': 143,
    'lansoprazole': 87.1,
    'valsartan': 112
}

# Check the column names in case you need to adjust them
print(filtered_data_with_SASA.columns)

# Iterate through the DataFrame and update SASA based on compound names
for index, row in filtered_data_with_SASA.iterrows():
    compound_name = row['solute_name']  # Replace with the actual column name containing the compound name
    if compound_name in compound_to_SASA and pd.isnull(row['SASA']):
        # Update the SASA value if the compound name matches and SASA is missing
        filtered_data_with_SASA.at[index, 'SASA'] = compound_to_SASA[compound_name]

# Save the updated DataFrame to a new CSV file
filtered_data_with_SASA.to_csv('filtered_data_with_SASA.csv', index=False)

Index(['solute_name', 'solute_smiles', 'solvent_name', 'solvent_smiles',
       'solute_inchikey', 'S_M', 'logS_M', 'MW', 'SASA'],
      dtype='object')


In [169]:
#add a column called compound that combines the solute_inchikey and solvent_name

# Load the filtered_data_with_SASA CSV file
filtered_data_with_SASA = pd.read_csv('filtered_data_with_SASA.csv')

# Create a new column 'compounds' that combines 'solute_inchikey' and 'solvent_name'
# Assuming solute_inchikey and solvent_name are the correct column names
filtered_data_with_SASA['Compound'] = filtered_data_with_SASA['solute_inchikey'].astype(str) + "_" + filtered_data_with_SASA['solvent_name'].astype(str)

# Reorder columns to make 'compounds' the first column
columns = ['Compound'] + [col for col in filtered_data_with_SASA.columns if col != 'Compound']
filtered_data_with_SASA = filtered_data_with_SASA[columns]

# Save the updated DataFrame to a new CSV file
filtered_data_with_SASA.to_csv('filtered_data_compound.csv', index=False)

# Print confirmation
print("The new document with 'Compound' column as the first column has been saved as 'filtered_data_compound.csv'.")


The new document with 'Compound' column as the first column has been saved as 'filtered_data_compound.csv'.


In [171]:
#merge the data files based on the 'compounds' column which can then give a final MNDO descriptors file

import pandas as pd

# Load the data files
filtered_data_compound = pd.read_csv('filtered_data_compound.csv')
MNDO_descriptors = pd.read_csv('MNDO_descriptors.csv')

# Merge the two DataFrames based on the 'compound' column
merged_data = pd.merge(filtered_data_compound, MNDO_descriptors, on='Compound', how='inner')

# Save the merged data to a new CSV file
merged_data.to_csv('MNDO_descriptors_final.csv', index=False)

# Print confirmation
print("The files have been successfully merged and saved as 'MNDO_descriptors_final.csv'.")


The files have been successfully merged and saved as 'MNDO_descriptors_final.csv'.


In [232]:
mndo_data = pd.read_csv('MNDO_descriptors_final.csv')

mndo_data = mndo_data.rename(columns={'logS_M': 'LogS'})
mndo_data = mndo_data.rename(columns={'volume': 'Volume'})

mndo_data.to_csv('MNDO_descriptors_final.csv', index=False)